In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.bayes_opt import bayesian_optimiser
from src.api_helper import api_utils
from src.api_helper import env
from datetime import datetime as dt, date as dt_date

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [5]:
@api_utils.multi_thread_transform
def api(query): return env.rosenbrock(query)  # computation on cpu

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query", x0); print(f"init reward {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu

init query tensor([[0.9705, 0.3561]])
init reward tensor([[-1.]]), origin reward: 34.32


##### OPT

In [17]:
# hyperparameters
T = 10  # total number of iterations

# gp; includes "MA2.5", "SE", "RQ", "LR", "PO"
gp_name, gp_params = "MA2.5",{
                          "mode": "raw",      # "raw", "add", "pro" for GPs
                          "opt":"quasi_newton",  # opt for MLE; (quasi_newton, ADAM)
                          "epochs":128,       # epoch to run, if chosen ADAM
                          "lr":1e-1,          # learning rate for ADAM
                         }
# q-parallelism (if use analytical acq_func, q must be 1)
batch_size = 1

acq_params = { 
    "acq_name" : "UCB",          # acqu func; includes: "EI", "UCB", "qEI", "qUCB", "qKG"
    "N_start": 32,               # number of starts for multi-start SGA
    "raw_samples" :512,          # heuristic initialisation 
    "N_MC_sample" : 256,         # number of samples for Monte Carlo simulation
    "num_fantasies": 128,        # number of fantasies used by KG
    "beta":1.,                   # used by UCB/qUCB
               }

bayes_opt = bayesian_optimiser(gp_name, gp_params, device, acq_params)

In [18]:
%%time
xs, ys = bayes_opt.outer_loop(T, (-4,4),x0, y0, r0, api, batch_size)  # maximising reward

Iter: 1, reward: -7.56
Iter: 2, reward: -0.91
Iter: 3, reward: -0.93
Iter: 4, reward: -0.96
Iter: 5, reward: -0.93
Iter: 6, reward: -0.94
Iter: 7, reward: -0.96
Iter: 8, reward: -0.95
Iter: 9, reward: -0.94
Iter: 10, reward: -0.95
CPU times: user 3.02 s, sys: 5.81 s, total: 8.83 s
Wall time: 2.27 s


In [16]:
xs.shape

torch.Size([41, 2])